<a href="https://colab.research.google.com/github/mluppichini/CleverRiver/blob/main/RiverPlumeIdentificationNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Configuration of Google Earth Engine

Execute the code to import the GEE libraries and then follow the Google procedure to configure correctly the API

In [ ]:
#@markdown Execute the Step 1 - Configuration Google Earth Engine

import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project="buoyant-site-331313")

import pandas as pd
import keras
import numpy as np



# Step 2: Import the python function and libraries

Execute the code to import the libraries and function used in this workflow.
Then this procedure import the data (see support material) into the "Support" folder

In [ ]:
#@markdown Execute the Step 2 - Import Functions and Libraries

from osgeo import gdal,osr
from osgeo.gdalconst import GDT_Float32
import gc
import tensorflow as tf
import geopandas as gp
import os
import ipywidgets as widgets
from IPython.display import display

try:
  os.mkdir("./Support")
except:
  pass

def LatLonImg(img, col, area, scale, NDSSI = False):
    img = img.addBands(ee.Image.pixelLonLat())

    img = img.reduceRegion(reducer=ee.Reducer.toList(), \
                           geometry=area, \
                           maxPixels=1e13, \
                           scale=scale);

    data = [np.array((ee.Array(img.get(col[i])).getInfo())) for i in range(len(col)) if col[i] != 'NDSSI']
    if NDSSI:
        print("calcolo NDSSI")
        b2 = np.array((ee.Array(img.get('B2')).getInfo()))
        b8 = np.array((ee.Array(img.get('B8')).getInfo()))
        ndssi = (b2 - b8) / (b8 + b2)
        data.append(ndssi)

    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data


def toImage(lats, lons, data):
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)  # -9999

    # fill the array with values
    counter = 0
    for y in range(0, len(arr), 1):
        for x in range(0, len(arr[0]), 1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(data) - 1:
                counter += 1
                arr[len(uniqueLats) - 1 - y, x] = data[counter]  # we start from lower left corner
    return arr

def saveDataFrameToGrid(df, xcol, ycol, value_col, n_crs, grid_output):
	#COSTRUISCO IL GRID
	row = df.iloc[0]
	row1 = df.iloc[1]

	rowLast = df.iloc[-1]
	rowLast1 = df.iloc[-2]
	#NUMERO DI RIGHE E NUMERO DI COLONNE
	rows = len(df[ycol].unique())
	cols = len(df[xcol].unique())
	cell_size_y = (df[ycol].max() - df[ycol].min())/rows
	cell_size_x = (df[xcol].max() - df[xcol].min())/cols
	x0, y0 = df[xcol].min() - cell_size_x/2, df[ycol].max()  + cell_size_y/2
	matrix = np.zeros((rows,cols), dtype=float)
	count = 0
	df = df.sort_values([ycol, xcol], ascending= [False, True])
	series = df[value_col].values
	for j in range(rows):
	    for i in range(cols):
	        matrix[j,i] = series[count]
	        count+=1
	x1,y1 = x0+cell_size_x*cols, y0 - cell_size_y*rows
	crs = osr.SpatialReference()
	crs.ImportFromEPSG(n_crs)
	output_grid = create_grid_xydiff(x0,x1,y1,y0,cell_size_x, cell_size_y,-9999, crs, grid_output)
	output_grid[3].WriteArray(matrix,0,0)

def create_grid_xydiff(xmin,xmax,ymin,ymax,cell_size_x,cell_size_y, NoData,sourceSR, output):
	#sourceSR = lyr.GetSpatialRef()
	#proj = osr.SpatialReference(wkt=raster.GetProjection())
    #crs = osr.SpatialReference()
    #crs.ImportFromEPSG(4326)
	cols= round( ((xmax - xmin)/float(cell_size_x)))
	rows= round(((ymax - ymin)/float(cell_size_y)))

	ext= output.split(".")[-1]

	if ext == "tif":
		driver = gdal.GetDriverByName('GTiff')
	elif ext == "tiff":
		driver = gdal.GetDriverByName('GTiff')
	elif ext == "flt":
		driver = gdal.GetDriverByName('EHdr')
	elif ext == "bt":
		driver = gdal.GetDriverByName('BT')
	else:
		print ("Definire un formato dell'output valido")
		return
	outDataset= driver.Create(output,cols,rows,1,GDT_Float32)
	gt= [xmin,cell_size_x,0,ymax,0,-cell_size_y]
	outDataset.SetGeoTransform(gt)

	# dst_wkt= spatialRef.ExportToWkt()
	#
	# outDataset.SetProjection(dst_wkt)
	outband= outDataset.GetRasterBand(1)
	outband.SetNoDataValue(NoData)
	# outRasterSRS = osr.SpatialReference()
	# outRasterSRS.ImportFromEPSG(3003)
	outDataset.SetProjection(sourceSR.ExportToWkt())
	out= [cols,rows,outDataset,outband]
	return out


def ApplyModel3(parametri):
    if  not os.path.isdir(os.path.join(parametri['simulations_out'], parametri['date_pred'])):
        os.mkdir(os.path.join(parametri['simulations_out'], parametri['date_pred']))

    tf.keras.backend.clear_session()
    keras.backend.clear_session()
    band_request = ['B%s' % i for i in range(1, 13)]
    band_request.append('B8A')

    NDSSI = False
    scale = 50

    # print ("Initialize DONE")
    xMin = parametri['bbox'][0]
    xMax = parametri['bbox'][2]
    yMin = parametri['bbox'][1]
    yMax = parametri['bbox'][3]

    area = ee.Geometry.Polygon([[xMin, yMin], \
                                [xMax, yMin], \
                                [xMax, yMax], \
                                [xMin, yMax], \
                                [xMin, yMin]])

    addTime = lambda x: x.set('Date', ee.Date(x.get('system:time_start')).format("YYYY-MM-dd"))
    collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
        .filterBounds(area) \
        .select(band_request) \
        .map(addTime) \
        .filter(ee.Filter.inList('Date', ee.List([parametri['date_pred']])))

    nImages = collection.size().getInfo()
    if nImages == 0: return
    imgs = collection.toList(collection.size())
    list_imgs = [ee.Image(imgs.get(i)) for i in range(nImages)]

    dates_unique = [parametri['date_pred']]
    dates_unique.sort()
    # check if I have to do a mosaic
    list_imgs = []
    dates = []
    for date in dates_unique:
        dates.append(date)
        date = ee.Date(date)
        filtered = collection.filterDate(date, date.advance(1, 'day'))
        image = ee.Image(filtered.mosaic())
        list_imgs.append(image)
    collection = ee.ImageCollection.fromImages(list_imgs)
    # self.nImages = collection.size().getInfo()
    valori = []

    for i, img in enumerate(list_imgs):
        date = dates[i]
        lat, lon, data = LatLonImg(img, band_request, area, scale, NDSSI)
        break
    lats = toImage(lat, lon, lat)
    lons = toImage(lat, lon, lon)
    index_b4 = band_request.index('B4')
    b4_data = toImage(lat, lon, data[index_b4])
    index_b2 = band_request.index('B2')
    b2_data = toImage(lat, lon, data[index_b2])
    index_b3 = band_request.index('B3')
    b3_data = toImage(lat, lon, data[index_b3])
    index_b8 = band_request.index('B8')
    b8_data = toImage(lat, lon, data[index_b8])
    model_path = os.path.join("./Support", "model.h5")
    model_path_json = os.path.join("./Support", "model.json")
    json_file = open(model_path_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = tf.keras.models.model_from_json(loaded_model_json)
    model.load_weights(model_path)
    dict_bands = {i: toImage(lat, lon, column_band) for i, column_band in enumerate(data)}
    nspace = int(parametri['model_setting']['nDim'] / 2)
    values_tot = []
    lats_effective, lons_effective, b4_values, b2_values, b3_values, b8_values = [], [], [], [], [], []
    for nrow in range(lats.shape[0]):
        for ncol in range(lats.shape[1]):
            values = []
            for nband, image in dict_bands.items():
                matrix_values = image[nrow - nspace: nrow + nspace + 1, ncol - nspace: ncol + nspace + 1]
                if matrix_values.shape != ( parametri['model_setting']['nDim'],  parametri['model_setting']['nDim']):
                    break
                values.append(matrix_values)
            values = np.array(values)
            if values.shape == (len(band_request),  parametri['model_setting']['nDim'],  parametri['model_setting']['nDim']):
                values_tot.append(values)
                lats_effective.append(lats[nrow][ncol])
                lons_effective.append(lons[nrow][ncol])
                b4_values.append(b4_data[nrow][ncol])
                b2_values.append(b2_data[nrow][ncol])
                b3_values.append(b3_data[nrow][ncol])
                b8_values.append(b8_data[nrow][ncol])
    keras.backend.clear_session()
    values_tot = np.array(values_tot)
    yhat = model.predict(values_tot, verbose=1)
    labels_predicted = [np.argmax(perc) for i, perc in enumerate(yhat)]
    perc_class0 = [perc[0] for i, perc in enumerate(yhat)]
    perc_class1 = [perc[1] for i, perc in enumerate(yhat)]
    perc_class2 = [perc[2] for i, perc in enumerate(yhat)]
    dfRes = pd.DataFrame()
    dfRes['x'] = lons_effective
    dfRes['y'] = lats_effective
    dfRes['value'] = labels_predicted
    dfRes['perc_class0'] = perc_class0
    dfRes['perc_class1'] = perc_class1
    dfRes['perc_class2'] = perc_class2
    dfRes['value_b4'] = b4_values
    dfRes['value_b2'] = b2_values
    dfRes['value_b3'] = b3_values
    dfRes['value_b8'] = b8_values
    dfRes.loc[dfRes['value'] == 2,'b4_mask'] = dfRes['value_b4']
    dfRes.loc[dfRes['value'] == 0,'b4_mask'] = 0
    dfRes.loc[dfRes['value'] == 2, 'b2_mask'] = dfRes['value_b2']
    dfRes.loc[dfRes['value'] == 0, 'b2_mask'] = 0
    dfRes.loc[dfRes['value'] == 2, 'b3_mask'] = dfRes['value_b3']
    dfRes.loc[dfRes['value'] == 0, 'b3_mask'] = 0
    dfRes.loc[dfRes['value'] == 2, 'b8_mask'] = dfRes['value_b8']
    dfRes.loc[dfRes['value'] == 0, 'b8_mask'] = 0
    dfRes['perc_class'] = dfRes['value_b4'] * dfRes['perc_class1']
    try:
        os.mkdir(os.path.join(parametri['simulations_out'],parametri['date_pred']))
    except: pass
    saveDataFrameToGrid(dfRes, 'x', 'y', 'b4_mask', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b4_mask.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'b2_mask', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b2_mask.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'b3_mask', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b3_mask.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'b8_mask', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b8_mask.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "mask.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value_b4', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b4_value.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value_b2', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b2_value.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value_b3', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b3_value.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value_b8', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "b8_value.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'perc_class', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "value_perc.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'perc_class0', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "perc_class0.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'perc_class1', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "perc_class1.tif" ))
    saveDataFrameToGrid(dfRes, 'x', 'y', 'value', 4326, os.path.join(parametri['simulations_out'],parametri['date_pred'], "classification.tif" ))

    del values_tot
    del dfRes
    del lats_effective, lons_effective, b4_values
    del lats, lons
    del data
    del yhat, labels_predicted, collection
    del model,
    keras.backend.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

# Step 3: Select the grid

Execute the code and from the generated select box chose the id of the rectangle of the grid that you want use.

In [ ]:
#@markdown Execute the Step 3 - Select from the Grid
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
dfGrid = gp.read_file("./Support/grid.shp")



# Crea il grafico
fig, ax = plt.subplots(figsize=(10, 10))
dfGrid.plot(ax=ax, color='blue', edgecolor='black')
dfGrid['centroid'] = dfGrid.geometry.centroid

# Aggiungi le etichette al centro dei poligoni
for x, y, label in zip(dfGrid.centroid.x, dfGrid.centroid.y, dfGrid['id']):
    ax.text(x, y, label, fontsize=10, ha='center', va='center')


values = dfGrid.id.values

select = widgets.Dropdown(
    options=values,
    value=values[0],
    description='ID Grid',
    disabled=False,
)

select

# Step 4: Select the dates and run the analyses images

Select the temporal period and execute the analysis of the images. The results are saved in in "Simulations" folder.

In [ ]:
#@markdown Execute the Step 4 - Apply the model

parametri = {}
id =select.value

geom = dfGrid[dfGrid.id == id].geometry.values[0]
bbox = geom.bounds
parametri['bbox'] = bbox
simulations_out = os.path.join('Simulations', str(id))
try:
  os.makedirs(simulations_out)
except: pass
parametri['simulations_out'] = simulations_out
parametri['model_setting'] = {"nNodi": 2 ** 5,'patience': 200,'nDim': 11}
date_start = '2018-03-22' # @param {type:"date"}
date_end = '2018-04-22' # @param {type:"date"}

date = date_start
i = 0
while date < date_end:
    date = pd.to_datetime(date_start) + pd.Timedelta(days=i)
    date = date.strftime('%Y-%m-%d')
    print (date)
    parametri['date_pred'] = date
    parametri['grid_number_execute'] = id
    ApplyModel3(parametri)
    tf.keras.backend.clear_session()
    keras.backend.clear_session()
    if len(os.listdir(os.path.join(parametri['simulations_out'],parametri['date_pred']))) == 0:
      os.rmdir(os.path.join(parametri['simulations_out'],parametri['date_pred']))
    i += 1



# Step 5: Create the zip file to export the results

In [ ]:
#@markdown Execute the Step 5 - Export Simulations folder
import shutil
shutil.make_archive("Simulations", 'zip', "./Simulations")